<b>Import all necessary items <b>

In [1]:
import pytesseract,requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
import xlsxwriter
import json
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

<b> JSON_DIR : specify path to json for subject data<br>
<b> EXCEL_DIR : specify output excel sheet path<br>
<b> USN_DIR : specify path to text file containing usns<br> 
    
<b>url1 : To get PHPSESSIONID and TOKEN<br>
<b> url2 : To get Captcha Image<br>
<b> url3 : To get Results<br>

<b> h1,h2 and h3 are the corresponding headers<br>
<b> payload : is the form data passed with url3<br>

In [2]:
JSON_DIR=r'./subject_data_7.json'
EXCEL_DIR=r'./k.xlsx'
USN_DIR=r'./k.txt'

url1='https://results.vtu.ac.in/_CBCS/index.php'
url2='https://results.vtu.ac.in/captcha/vtu_captcha.php'
url3='https://results.vtu.ac.in/_CBCS/resultpage.php'

h1={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Host': 'results.vtu.ac.in',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'none',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

h2={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
'Accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-Mode': 'no-cors',
'Referer': 'https://results.vtu.ac.in/_CBCS/index.php',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
'Cookie': 'PHPSESSID=4u006bkqm75chfavdckmpebna6'}

h3={
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
'Cache-Control': 'no-cache',
'Connection': 'keep-alive',
'Content-Length': '80',
'Content-Type': 'application/x-www-form-urlencoded',
'Cookie': 'PHPSESSID=4u006bkqm75chfavdckmpebna6',
'Host': 'results.vtu.ac.in',
'Origin': 'https://results.vtu.ac.in',
'Pragma': 'no-cache',
'Referer': 'https://results.vtu.ac.in/_CBCS/index.php',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

payload={'Token':'29822fda6dcdb89893834f2584792e18754f2033',
'lns':'1BI16CS001',
'captchacode':'52984'}

<b>To reset and get new session and captcha

In [3]:
def reset():
    r1 = requests.get(url1, headers=h1,verify=False)
    
    h2['Cookie']=r1.headers['Set-Cookie'].rstrip('; path=/')
    h3['Cookie']=r1.headers['Set-Cookie'].rstrip('; path=/')
    soup = BeautifulSoup(r1.text,'html.parser')
    
    token = soup.find_all('input')[0]['value']
    payload['Token']=token
    
    k=str(soup.find_all('img')[1]['src'])
    params={'_CAPTCHA':'','t':k[36:]}

    r2 = requests.get(url2, headers=h2,params=params,verify=False)

    im = Image.open(BytesIO(r2.content))

    #im.save(r'C:\Users\SAMKIT~1\Desktop\k.jpg')
    pix=im.load()
    w,h=im.size

    for x in range(0,w):
        for y in range(0,h):
                r,g,b=pix[x,y]
                if r==g==b==102:
                        pix[x,y]=(0,0,0)
                else:
                        pix[x,y]=(255,255,255)

    im.save(r'C:\Users\SAMKIT~1\AppData\Local\Temp\k.jpg')
    cap=pytesseract.image_to_string(Image.open(r'C:\Users\SAMKIT~1\AppData\Local\Temp\k.jpg'))
    payload['captchacode']=cap

<b>Grade function converts Marks to Grade<br>
<b>If Fail then Grade is 0

In [4]:
def grade(marks,pf):
    if pf!='P':
        fail=True
        return 0
    marks=int(marks)
    if marks>=90 and marks<=100:
        return 10
    elif marks>=80 and marks<90:
        return 9
    elif marks>=70 and marks<80:
        return 8
    elif marks>=60 and marks<70:
        return 7
    elif marks>=50 and marks<60:
        return 6
    elif marks>=45 and marks<50:
        return 5
    else:
        return 4

In [5]:
def calTotalCredits(sub_data):
    sum=0
    for sub in sub_data:
        sum=sum+sub_data[sub]['credits']
    return sum

In [6]:
def main():
    with open(JSON_DIR) as f:
      sub_data = json.load(f)
    total_credits=calTotalCredits(sub_data)
    data=xlsxwriter.Workbook(EXCEL_DIR)
    sheet1=data.add_worksheet()
    sheet1.write(0,0,"USN")
    sheet1.write(0,1,"NAME")
    cols=2
    for sub in sub_data:
        if len(sub_data[sub]['scode'])==1:
            sheet1.write(0,cols,sub_data[sub]['scode'][0])
            sheet1.write(1,cols,'IA')
            sheet1.write(1,cols+1,'EXT')
            sheet1.write(1,cols+2,'TOTAL')
            sheet1.write(1,cols+3,'GRADE')
            cols+=4
        else:
            sheet1.write(0,cols,"Elective")
            sheet1.write(1,cols,'CODE')
            sheet1.write(1,cols+1,'IA')
            sheet1.write(1,cols+2,'EXT')
            sheet1.write(1,cols+3,'TOTAL')
            sheet1.write(1,cols+4,'GRADE')
            cols+=5
    sheet1.write(1,cols,'SGPA')
    
    reset()
    
    with open(USN_DIR,'r') as fh:
        #r is Row
        r=2
        for usn in fh:
            try:
                fail=False
                #cp is credit points
                cp=0
                usn=usn.strip()
                payload['lns']=str(usn)
                r3 = requests.post(url3,headers=h3,data=payload,verify=False)
                while "Invalid captcha code !!!" in r3.text:
                    reset()
                    r3 = requests.post(url3,headers=h3,data=payload,verify=False)
                if "Redirecting to VTU Results Site" in r3.text:
                    reset()
                    r3 = requests.post(url3,headers=h3,data=payload,verify=False)
    
                soup = BeautifulSoup(r3.text,'html.parser')
                usn=soup.find_all('td')[1].text.lstrip(' : ')
                name=soup.find_all('td')[3].text.lstrip(' : ')
                sem=soup.find_all('b')[6].text[-1]
                print('NAME : %s\tUSN : %s\tSEM : %s'%(name,usn,sem))
                sheet1.write(r,0,usn)
                sheet1.write(r,1,name)
                
                odiv=soup.find_all('div',{'class':'divTableRow'})
                c=2
                for sub in sub_data:
                    scode=sub_data[sub]['scode']
                    credits=sub_data[sub]['credits']
                    total_marks=sub_data[sub]['total marks']    
                    scale=100/total_marks
                    mult_codes=len(sub_data[sub]['scode'])!=1
                    
                    sub_found=False
                    nos=len(odiv)
                    for i in range(1,nos-1):
                        div=odiv[i].find_all('div')
                        if div[0].text in scode:
                            k=str(div[0].text)+'\t'+str(div[4].text)+'\t'+str(div[5].text)
                            print(k)
                            sub_found=True
                            code=div[0].text
                            ia=div[2].text
                            ext=div[3].text
                            total=int(div[4].text)*scale
                            pf=div[5].text
                            if pf=='F':
                                fail=True
                    if sub_found==False:
                        break
                    #print(ia+'\n'+ext+'\n'+str(total)+'\n'+code+'\n')
                    gr=grade(total,pf)
                    cp+=gr*credits
                    if mult_codes:
                        sheet1.write(r,c,code)
                        c+=1
                    sheet1.write(r,c,ia)
                    sheet1.write(r,c+1,ext)
                    sheet1.write(r,c+2,total)
                    sheet1.write(r,c+3,gr)
                    c+=4
                        
                if not fail:
                    sheet1.write(r,c,round(cp/total_credits,2))
                r+=1
            except Exception as e:
                with open('invalid.txt','a') as ifh:
                    ifh.writelines(usn)
                continue    
    data.close()

In [ ]:
if __name__=="__main__":
    main()